<a href="https://colab.research.google.com/github/Emirismail/DeepL-journey/blob/master/Starter/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TensorBoard for google colab
! wget https://raw.githubusercontent.com/hse-aml/intro-to-dl/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab

# run tensorboard in background
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# expose port and show the link
setup_google_colab.expose_port_on_colab(6006)

## Warm UP

### What is TF?

TensorFlow is a deep learning framework
TF is essentially 2 things:


1.   A tool to describe computational graphs: 
      *   Graph object --> Network of nodes 
2.   A runtime for execution of these graphs:
      *   CPU, GPU, TPU
      *   One mode or distributed mode







### Inputs



*   Placeholder:
    * Placeholder for a tensor that we dont have it right now
    * Features (size & type) 



In [0]:
features= tf.placeholder(tf.float32, shape= (None,D))
target= tf.placeholder(tf.float32, shape= (None,1))

*   Variable:
    * Tensor with some value
    * Weights matrix

In [0]:
x= tf.get_variable("x", shape= (), dtype=tf.float32, trainable= True)

*   Constant:
    * This is a tensor with a value.

### Computational Graph



*   After importing, tf creates a default graph
*   We can get it with:



In [0]:
tf.get_default_graph()

*   We can create our own graph:



In [0]:
g= tf.Graph()
with g.as_default():
  pass

*   We can clear the default with:



In [0]:
tf.reset_default_graph()

### Operations & Tensors

*   Every node in our graph is an operation
*   We can list nodes with:



In [0]:
tf.get_default_graph().get_operations()

*  Get output tensors: 



In [0]:
tf.get_default_graph().get_operations()[0].outputs 

### Running Graph

We need an other object
*   A tf.session object
*   Create a session :



In [0]:
s= tf.InteractiveSession()

*   Define the graph :



In [0]:
x= tf.constant(2.0)
y= tf.constant(5.0)
w= x*y



*   Run the graph :


In [0]:
print(w)
print(s.run(w)) # The result of computation

### Why we need a session object?
*   We will define our graph in Python, but these operations will not be executed in Python (Very slow)
*   Operations are written in C++ to run on CPU, GPU, TPU
*   Session object will own necessary resources to execute our graph
*   It is important to release these resources with:



In [0]:
tf.Session.close()

## Simplest Trial

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.15.2




*   We will optimize the function f= x**2 with respect to x



In [24]:
tf.reset_default_graph()
x= tf.get_variable("x", shape= (), dtype=tf.float32, trainable= True)
#y= tf.get_variable("y", shape= (), dtype=tf.float32, trainable= True)
f= x**2
f= tf.Print(f, [x, f], "x, f:")

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:



In [25]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [0]:
optimizer= tf.train.GradientDescentOptimizer(0.1)
step= optimizer.minimize(f, var_list= [x])

In [0]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries= tf.summary.merge_all()

In [28]:
s= tf.InteractiveSession()
summary_writer= tf.summary.FileWriter("logs/1", s.graph)
s.run(tf.global_variables_initializer())

/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
for i in range(10):
  _, curr_x, curr_f= s.run([step, x, f])
  print(curr_x, curr_f)

0.32939425 0.16953214
0.2635154 0.10850057
0.21081233 0.06944037
0.16864987 0.044441838
0.1349199 0.028442778
0.10793592 0.018203378
0.086348735 0.011650163
0.06907899 0.007456104
0.05526319 0.004771907
0.044210553 0.0030540202


In [0]:
for i in range(10):
  s.run([step,f])

In [0]:
for i in range(10):
  _, curr_summaries= s.run([step, summaries])
  summary_writer.add_summary(curr_summaries, i)
  summary_writer.flush()

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs

## Linear Regression

In [0]:
N= 1000
D= 3
x= np.random.random((N,D))
w= np.random.random((D,1))
y= x@w + np.random.randn(N,1)*0.2

In [0]:
tf.reset_default_graph()
features= tf.placeholder(tf.float32, shape= (None,D))
target= tf.placeholder(tf.float32, shape= (None,1))

In [0]:
weights= tf.get_variable("w", shape= (D,1), dtype=tf.float32, trainable= True)
predictions= features@weights

In [0]:
loss= tf.reduce_mean((target-predictions)**2)

In [0]:
optimizer= tf.train.GradientDescentOptimizer(0.1)
step= optimizer.minimize(loss)

In [34]:
s= tf.InteractiveSession()
# Model checkpoints

saver= tf.train.Saver(tf.trainable_variables())
s.run(tf.global_variables_initializer())

/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [35]:
for i in range(300):
  _, curr_loss, curr_weights= s.run(
      [step, loss, weights], 
      feed_dict={features:x, target:y}
  )
  if i%50==0:
    saver.save(s, "logs/3/model.ckpt", global_step=i)
    print(curr_loss)

0.9635797
0.061085388
0.040496565
0.03674502
0.036059238
Instructions for updating:
Use standard file APIs to delete files with this prefix.
0.035933442


In [15]:
# found weights
curr_weights.T

[[0.22669326]
 [0.07311469]
 [0.34474435]]


In [16]:
# true weights
w.T

[[0.25578636]
 [0.06338396]
 [0.33437311]]


In [36]:
saver.last_checkpoints

['logs/3/model.ckpt-50',
 'logs/3/model.ckpt-100',
 'logs/3/model.ckpt-150',
 'logs/3/model.ckpt-200',
 'logs/3/model.ckpt-250']

In [37]:
saver.restore(s, "logs/3/model.ckpt-150")

INFO:tensorflow:Restoring parameters from logs/3/model.ckpt-150


## Takeaway

*   TF has built-in optimizers that do back-propagation automatically.
*   TBoard provides tools for visualization:
    *   Training process
    *   Loss function
*   TF allows to checkpoint the graph to restore its state later.

